# LLM-rankers on PLAID-X

This jupyter notebook uses [llm-rankers](https://github.com/ielab/llm-rankers) to re-rank the top-100 documents retrieved by [plaid-x](https://github.com/hltcoe/ColBERT-X) in the context [ReNeuIR 2024](https://reneuir.org/). Please look at the [corresponding publications](https://github.com/ielab/llm-rankers#references) for more details.

### Step 1: Import dependencies

In [2]:
import torch
from tqdm import tqdm
import pandas as pd
from tira.third_party_integrations import ir_datasets, persist_and_normalize_run
from tira.rest_api_client import Client
from llmrankers.setwise import SetwiseLlmRanker
from llmrankers.rankers import SearchResult
import os


### Step 2: Load and Transform the Dataset

In [16]:
# the "from tira.third_party_integrations import ir_datasets" import patches "ir_datasets.load"
# so that it loads the dataset injected into the tira sandbox when executed within the sandbox.
# I.e., we only ensure that it runs on a minimal spot-check dataset here.
dataset_id = 'reneuir-2024/dl-top-10-docs-20240701-training'
dataset = ir_datasets.load(dataset_id)

queries = {i.query_id: i.default_text() for i in dataset.queries_iter()}
docs_store = dataset.docs_store()

Download: 3.60MiB [00:00, 8.07MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/reneuir-2024/dl-top-10-docs-20240701-training/


In [18]:
tira = Client()
plaid_x = tira.get_run_output('reneuir-2024/reneuir-baselines/plaid-x-retrieval', dataset_id)

# we have no pyterrier in this docker image, so we load the previous stage via pandas
plaid_x = pd.read_csv(plaid_x + '/run.txt', sep="\s+", names=["qid", "q0", "docid", "rank", "score", "system"], dtype={"qid": str, "docid": str})

# filter to top-100 results
plaid_x = plaid_x[plaid_x["rank"] <= 100]

qid_to_rerank_data = {}

for _, i in tqdm(plaid_x.iterrows(), 'Transform data'):
    if i['qid'] not in qid_to_rerank_data:
        qid_to_rerank_data[i['qid']] = {'query': queries[i['qid']], 'search_results': []}

    qid_to_rerank_data[i['qid']]['search_results']  += [
        SearchResult(docid=i['docid'], text=docs_store.get(i['docid']).default_text(), score=i['score'])
    ]

Transform data: 9700it [00:00, 40897.70it/s]


### Step 3: Re-Rank

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Will use device:', device)

# We make the model injectable into the notebook so that we can run multiple models with the same code.
model = os.environ.get('MODEL', 'google/flan-t5-small')

print('Model is:', model)

ranker = SetwiseLlmRanker(model_name_or_path=model, tokenizer_name_or_path=model, device=device)

print('Ranker is', ranker)

In [ ]:
run = []

for qid in tqdm(qid_to_rerank_data):
    query = qid_to_rerank_data[qid]['query']
    search_results = qid_to_rerank_data[qid]['search_results']

    for i in ranker.rerank(query, search_results):
        run += [{"qid": qid, "score": i.score, "docno": i.docid}]


### Step 4: Persist run file

In [ ]:
run = pd.DataFrame(run)

persist_and_normalize_run(run, system_name=f'llm-rankers-plaid-x-{model}', default_output='.')